In [5]:
!pip install -r https://raw.githubusercontent.com/anujavenkatachalam04/chvi_vbd_rj/main/requirements.txt

In [4]:
import os
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
import geopandas as gpd
from shapely.geometry import Point
import uuid


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
os.chdir("/content/drive/MyDrive/CHVI")


In [7]:
blocks_shp=gpd.read_file("5_Shapefiles/SUBDISTRICT_11/Rajasthan_Blocks.geojson")

In [8]:
grid=gpd.read_file("5_Shapefiles/Rajasthan_10kmx10km_grid.geojson")

In [9]:
import os
import pandas as pd
import requests_cache
from retry_requests import retry
from dotenv import load_dotenv
import openmeteo_requests

# Load API key from .env
load_dotenv()
OPENMETEO_API_KEY = os.getenv("OPENMETEO_API_KEY")

In [11]:
OPENMETEO_API_KEY

# Extract ERA5 data at 10km x 10km resolution

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm

start_date = "2024-01-01"
end_date = "2025-09-30"

base_url = "https://archive-api.open-meteo.com/v1/era5"

# Convert list fields to comma-separated strings as required by API
params = {

    "daily": [
      "temperature_2m_max", "temperature_2m_min",
      ""
     "relative_humidity_2m_mean", "precipitation_sum"
    ],
    "timezone": "Asia/Kolkata",
    "start_date": start_date,
    "end_date": end_date,
    "apikey": "FrBHTVq25VDS9Lfu"
}

all_data = []
session = requests.Session()  # Reuse TCP connection

for idx, row in tqdm(grid_clipped.iterrows(), total=len(grid_clipped)):
    lat, lon = row.geometry.y, row.geometry.x

    try:
        response = session.get(base_url, params={**params, "latitude": lat, "longitude": lon}, timeout=10)
        response.raise_for_status()  # Raise exception for HTTP errors

        data = response.json()
        daily = data.get("daily")

        if daily:
            daily_df = pd.DataFrame(daily)
            daily_df["latitude"] = lat
            daily_df["longitude"] = lon
            daily_df["point_id"] = idx
            daily_df["DISTRICT"] = row["DISTRICT"]
            daily_df["NAME"] = row["NAME"]
            all_data.append(daily_df)
        else:
            print(f"No daily data for point {idx} ({lat}, {lon})")

    except Exception as e:
        print(f"Request failed for point {idx} ({lat}, {lon}): {e}")

session.close()

# Combine all data
if all_data:
    weather_df = pd.concat(all_data, ignore_index=True)
else:
    weather_df = pd.DataFrame()


100%|██████████| 3420/3420 [13:01<00:00,  4.37it/s]


In [ ]:
weather_df.to_csv("climate_data/openmeteo_era5_grid_data_10kmx10km_2024-01-01_to_2025-09-30.csv", index=False)

In [ ]:
def get_season(month):
    if month in [3, 4, 5]:
        return 'Summer (Mar-May)'
    elif month in [6, 7, 8]:
        return 'Monsoon (Jun-Aug)'
    elif month in [9, 10, 11]:
        return 'Post-Monsoon (Sep-Nov)'
    else:
        return 'Winter (Dec-Feb)'

In [ ]:
all_weather.drop(columns=["month_num"], inplace=True)

In [ ]:
all_weather['season'] = all_weather["time"].dt.month.apply(get_season)

In [ ]:
all_weather["diurnal_temperature_range"]=all_weather["temperature_2m_max"]-all_weather["temperature_2m_min"]

In [ ]:
all_weather.to_csv("climate_data/openmeteo_era5_grid_data_10kmx10km_2019-01-01_to_2025-06-29.csv", index=False)

# Climate - Aggregate API data (at grid-level) to block & day

In [ ]:
all_weather=pd.read_csv("climate_data/openmeteo_era5_grid_data_10kmx10km_2019-01-01_to_2025-06-29.csv")

In [ ]:
all_weather["time"]=pd.to_datetime(all_weather["time"])

In [ ]:
group_cols = ["dtname", "sdtname", "time"]

# Compute mean values per block and date
block_daily_climate = all_weather.groupby(group_cols).agg(
    {"temperature_2m_mean":"mean",
     "temperature_2m_max":"mean",
     "temperature_2m_min":"mean",
     "relative_humidity_2m_mean":"mean",
     "rain_sum":"sum",
     "diurnal_temperature_range":"mean"}
).reset_index()

In [ ]:
block_daily_climate.columns

Index(['dtname', 'sdtname', 'time', 'temperature_2m_mean',
       'temperature_2m_max', 'temperature_2m_min', 'relative_humidity_2m_mean',
       'rain_sum', 'diurnal_temperature_range'],
      dtype='object')

In [ ]:
block_daily_climate["time"].min(), block_daily_climate["time"].max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2025-06-29 00:00:00'))

In [ ]:
block_daily_climate["season"]=block_daily_climate["time"].dt.month.apply(get_season)

In [ ]:
block_daily_climate["season"].value_counts(normalize=True)

,proportion
season,
Summer,0.271501
Winter,0.253373
Monsoon,0.244941
Post-Monsoon,0.230185


In [ ]:
block_daily_climate.to_csv("dengue_malaria/clean/daily_climate_block.csv", index=False)

# Climate - Aggregate API data (at grid-level) to district & day

In [ ]:
all_weather=pd.read_csv("climate_data/openmeteo_era5_grid_data_10kmx10km_2019-01-01_to_2025-06-29.csv")

In [ ]:
all_weather["time"]=pd.to_datetime(all_weather["time"])

In [ ]:
agg_district = (
    all_weather.groupby(['dtname', 'time'])
    .agg({
        'temperature_2m_mean': 'mean',
        'relative_humidity_2m_mean': 'mean',
        'temperature_2m_max': 'mean',
        'temperature_2m_min': 'mean',
        'rain_sum': 'sum',
        'diurnal_temperature_range': 'mean'
    })
    .reset_index()
)


In [ ]:
agg_district["time"].min(), agg_district["time"].max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2025-06-29 00:00:00'))

In [ ]:
agg_district["season"]=agg_district["time"].dt.month.apply(get_season)

In [ ]:
agg_district["season"].value_counts(normalize=True)

,proportion
season,
Summer,0.271501
Winter,0.253373
Monsoon,0.244941
Post-Monsoon,0.230185


In [ ]:
agg_district.columns

Index(['dtname', 'time', 'temperature_2m_mean', 'relative_humidity_2m_mean',
       'temperature_2m_max', 'temperature_2m_min', 'rain_sum',
       'diurnal_temperature_range', 'season'],
      dtype='object')

In [ ]:
agg_district.to_csv("dengue_malaria/clean/daily_climate_district.csv", index=False)

In [ ]:
# The End!